In [1]:
!pip install torch torchvision numpy matplotlib tqdm


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bar


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

# ✅ Define the TinyNeRF model
class TinyNeRF(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=128):
        super(TinyNeRF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 4)  # 3 for RGB, 1 for density

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# ✅ Function to sample rays
def sample_rays(height, width, focal, c2w):
    """Generate ray origins and directions."""
    i, j = torch.meshgrid(torch.arange(width), torch.arange(height), indexing="ij")
    dirs = torch.stack([(i - width * 0.5) / focal, -(j - height * 0.5) / focal, -torch.ones_like(i)], dim=-1)
    rays_d = torch.sum(dirs[..., None, :] * c2w[:3, :3], dim=-1)  # Rotate by camera pose
    rays_o = c2w[:3, 3].expand(rays_d.shape)
    return rays_o, rays_d

# ✅ Volume Rendering Function (Fixed)
def volume_rendering(model, rays_o, rays_d, num_samples=64):
    """Sample points along rays and render an image using NeRF."""
    t_vals = torch.linspace(0., 1., steps=num_samples).to(rays_o.device)  # Depth samples
    points = rays_o[..., None, :] + rays_d[..., None, :] * t_vals[..., None]  # (H, W, S, 3)

    view_dirs = rays_d / torch.norm(rays_d, dim=-1, keepdim=True)  # Normalize directions
    view_dirs = view_dirs[..., None, :].expand(points.shape)  # Match shape

    points = points.reshape(-1, 3)  # Flatten (N, 3)
    view_dirs = view_dirs.reshape(-1, 3)  # 🔹 FIXED: Using `reshape()` instead of `view()`

    inputs = torch.cat([points, view_dirs], dim=-1)  # Concatenate xyz + view dir (N, 6)

    outputs = model(inputs)  # Get RGB + density
    colors, sigma = torch.sigmoid(outputs[..., :3]), F.relu(outputs[..., 3])
    
    return colors.view(rays_o.shape[0], rays_o.shape[1], num_samples, 3)  # Reshape output

# ✅ Initialize Model and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyNeRF().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# ✅ Set up synthetic data
height, width, focal = 100, 100, 50
c2w = torch.eye(4)  # Dummy camera pose

# ✅ Training Loop
num_epochs = 1000
for epoch in tqdm(range(num_epochs)):
    rays_o, rays_d = sample_rays(height, width, focal, c2w)
    colors_gt = torch.rand(height, width, 3).to(device)  # Dummy ground truth
    colors_pred = volume_rendering(model, rays_o.to(device), rays_d.to(device))

    loss = criterion(colors_pred.mean(dim=2), colors_gt)  # Compare mean color
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# ✅ Save trained model
torch.save(model.state_dict(), "tinynrf.pth")
print("Model saved as tinynrf.pth")


  0%|                                                                                 | 1/1000 [00:02<48:48,  2.93s/it]

Epoch 0: Loss = 0.08392017334699631


 10%|███████▉                                                                       | 101/1000 [04:17<39:03,  2.61s/it]

Epoch 100: Loss = 0.082729771733284


 20%|███████████████▉                                                               | 201/1000 [08:30<34:36,  2.60s/it]

...

Epoch 800: Loss = 0.08347784727811813


 90%|███████████████████████████████████████████████████████████████████████▏       | 901/1000 [23:56<01:48,  1.10s/it]

Epoch 900: Loss = 0.0833701640367508


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [25:41<00:00,  1.54s/it]

Model saved as tinynrf.pth
